In [0]:
from pyspark.sql.functions import *
#Azure Event Hub Configuration

event_hub_namespace="hub-namespace"
event_hub_name="<<hub-name>>"
event_hub_connection_string="<<connection string>>"
# hospitalanalyticsvaultscope
kafka_options={
    'kafka.bootstrap.servers':f"{event_hub_namespace}:9093",
    'subscribe':event_hub_name,
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.jaas.config': f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{event_hub_connection_string}\";",
    'startingOffsets':'earliest',
    'failOnDataLoss':'false'
}

#read data from event hub
raw_df=(
    spark.readStream.format("kafka").options(**kafka_options).load()
)

#cast data to json
json_df=raw_df.selectExpr("CAST(value as string) as raw_json")


# ADLS Configuration
spark.conf.set(
    "fs.azure.account.key.<<storage account name>>.dfs.core.windows.net",
    dbutils.secrets.get(scope="hospitalanalyticsvaultscope",key="storage-connection")
)

bronze_path="abfss://bronze@<<storage account name>>.dfs.core.windows.net/bronze/patient_flow"
silver_path="abfss://silver@<<storage account name>>.dfs.core.windows.net/silver/"


# write stream to bronze
(
    json_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation","abfss://bronze@<<storage account name>>.dfs.core.windows.net/_checkpoints/patient_flow")
    .start(bronze_path)
)

In [0]:
import json

manual_json={"patient_id": "d44ac24b-4afe-470c-b576-12e7a8421823", "gender": "Female", "age": 64, "department": "Maternity", "admission_time": "2026-02-10T03:59:37.243612+00:00", "discharge_time": "2026-02-07T06:59:37.242690+00:00", "bed_id": 235, "hospital_id": 4}

manual_df=spark.createDataFrame([(json.dumps(manual_json),)],["raw_json"])
manual_df.write.format("delta").mode("append").save(bronze_path)

In [0]:
display(spark.read.format("delta").load(bronze_path))

raw_json
"{""patient_id"": ""d44ac24b-4afe-470c-b576-12e7a8421823"", ""gender"": ""Female"", ""age"": 64, ""department"": ""Maternity"", ""admission_time"": ""2026-02-10T03:59:37.243612+00:00"", ""discharge_time"": ""2026-02-07T06:59:37.242690+00:00"", ""bed_id"": 235, ""hospital_id"": 4}"
"{""patient_id"": ""a4227592-001b-4e7a-bf42-eb9734287b37"", ""gender"": ""Female"", ""age"": 141, ""department"": ""Pediatrics"", ""admission_time"": ""2026-02-07T13:26:42.628223+00:00"", ""discharge_time"": ""2026-02-09T23:26:42.628223+00:00"", ""bed_id"": 106, ""hospital_id"": 4}"
"{""patient_id"": ""d44ac24b-4afe-470c-b576-12e7a8421823"", ""gender"": ""Female"", ""age"": 94, ""department"": ""Maternity"", ""admission_time"": ""2026-02-10T03:59:37.243612+00:00"", ""discharge_time"": ""2026-02-07T06:59:37.242690+00:00"", ""bed_id"": 235, ""hospital_id"": 4}"
"{""patient_id"": ""c52add2d-76ad-42b5-ab99-023f7cdbd52c"", ""gender"": ""Female"", ""age"": 105, ""department"": ""Cardiology"", ""admission_time"": ""2026-02-07T03:23:21.901160+00:00"", ""discharge_time"": ""2026-02-09T05:23:21.901160+00:00"", ""bed_id"": 310, ""hospital_id"": 5}"
"{""patient_id"": ""a5d8ce5d-e58a-494e-9165-08874dafd3ac"", ""gender"": ""Male"", ""age"": 77, ""department"": ""Cardiology"", ""admission_time"": ""2026-02-08T07:52:45.556768+00:00"", ""discharge_time"": ""2026-02-10T23:52:45.556768+00:00"", ""bed_id"": 447, ""hospital_id"": 7}"
"{""patient_id"": ""a685d218-9c1f-4e15-9687-2e3305a9a037"", ""gender"": ""Female"", ""age"": 137, ""department"": ""Cardiology"", ""admission_time"": ""2026-02-06T08:01:48.519709+00:00"", ""discharge_time"": ""2026-02-08T21:01:48.519709+00:00"", ""bed_id"": 302, ""hospital_id"": 5}"
"{""patient_id"": ""0552bac8-d984-43a8-84c7-180a17e6a6f3"", ""gender"": ""Female"", ""age"": 127, ""department"": ""Cardiology"", ""admission_time"": ""2026-02-06T21:20:22.924863+00:00"", ""discharge_time"": ""2026-02-09T12:20:22.924863+00:00"", ""bed_id"": 263, ""hospital_id"": 2}"
"{""patient_id"": ""72b02c9e-7062-493d-8182-e1b9405d6c6f"", ""gender"": ""Female"", ""age"": 114, ""department"": ""Pediatrics"", ""admission_time"": ""2026-02-06T08:18:49.149200+00:00"", ""discharge_time"": ""2026-02-07T13:18:49.149200+00:00"", ""bed_id"": 500, ""hospital_id"": 6}"
"{""patient_id"": ""5505598f-5af4-4b74-8351-d686fb1b2aad"", ""gender"": ""Female"", ""age"": 110, ""department"": ""Pediatrics"", ""admission_time"": ""2026-02-07T04:07:18.957026+00:00"", ""discharge_time"": ""2026-02-09T21:07:18.957026+00:00"", ""bed_id"": 129, ""hospital_id"": 6}"
"{""patient_id"": ""95e02857-7f14-44a5-b15b-5475cce99df1"", ""gender"": ""Female"", ""age"": 148, ""department"": ""Cardiology"", ""admission_time"": ""2026-02-07T22:15:02.312341+00:00"", ""discharge_time"": ""2026-02-09T20:15:02.312341+00:00"", ""bed_id"": 349, ""hospital_id"": 4}"
